## 💸 Stock Trend Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [226]:
# !pip install -r requirements.txt

In [227]:
import yfinance as yf
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, dcc, html, Input, Output, State
import json
from io import StringIO
import ast

Initialization of Analysis Parameters

In [228]:
TICKERS: list[str] = ["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "JNJ", "JPM", "XOM", "PG", "WMT", "AMT"]
START_DATE: str = "2022-01-01"
END_DATE: str = "2025-07-18"

ma_period: list[int] = [50, 200]
vol_period: int = 20
rsi_period: int = 14
macd_short_period: int = 12
macd_long_period: int = 26
macd_signal_period: int = 9

macd_windows: list[dict[str, str]] = [
    {'label': 'Classic (12, 26, 9)', 'value': '(12, 26, 9)'},
    {'label': 'Short-term (5, 35, 5)', 'value': '(5, 35, 5)'},
    {'label': 'Long-term (19, 39, 9)', 'value': '(19, 39, 9)'},
]

### 🏗️ Data Acquisition

Function for Fetching Daily Price Data

In [229]:
def get_stock_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with date-indexed adjusted closing price data
    """
    
    data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance and select relevant columns
            df = yf.download(ticker, start=start, end=end, auto_adjust=True)[['Close', 'Volume']]
            df.columns = ['Close', 'Volume']
            
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data found for {ticker} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

### 🧹 Data Cleaning and Preparation

Function for Filling Missing Values and Normalizing Values

In [230]:
def clean_data(data: dict[str, pd.DataFrame]) -> dict:
    """
    Cleans and prepares the stock data for analysis.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with cleaned and normalized data
    """

    cleaned_data = {}

    for ticker, df in data.items():
        # Replace zeros in 'Close' with NaN
        df.loc[df['Close'] == 0, 'Close'] = np.nan

        # Forward and backward fill missing values
        df = df.ffill()
        
        # Normalize the 'Close' prices to a range of 0 to 1
        df['Close_Normalized'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())

        df = df[['Close', 'Close_Normalized', 'Volume']] # Change column order
        
        cleaned_data[ticker] = df

    return cleaned_data

### 📊 Data Analysis

Functions for Calculating Return, Moving Average, and Volatility

In [231]:
def calculate_return(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the daily return of the stock based on the 'Close' price.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices

    Returns:
        pd.DataFrame: DataFrame with an additional 'Return' column
    """
    
    df['Return'] = df['Close'].pct_change()
    return df

def calculate_moving_average(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the moving average of the 'Close' price over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        window (int): The number of periods over which to calculate the moving average
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the moving average
    """

    df[f'Moving_Avg_{window}'] = df['Close'].rolling(window=window).mean()
    return df

def calculate_volatility(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the volatility of the stock based on the 'Return' column over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Return' column
        window (int): The number of periods over which to calculate the volatility
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the volatility
    """

    df['Rolling_Vol'] = df['Return'].rolling(window=window).std()
    return df

Function for Calculating all the Measures

In [232]:
def calculate_measures(data: dict[str, pd.DataFrame], ma_window: int | list[int] = ma_period, vol_window: int = vol_period) -> dict:
    """
    Calculates the return, moving average and the volatility for each stock's closing price.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols
        ma_window (int | list[int]): Window size(s) for moving average (Default to 50)
        vol_window (int): Window size for rolling volatility calculation

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with volatility added
    """
    
    # Housekeeping
    if isinstance(ma_window, int):
        ma_window = [ma_window]

    ma_period = ma_window
    vol_period = vol_window

    final_data = {}

    for ticker, df in data.items():
        df = calculate_return(df)
        for ma_w in ma_period:
            df = calculate_moving_average(df, ma_w)
        df = calculate_volatility(df, vol_period)

        final_data[ticker] = {
            'data': df,
            'overall_vol': float(df['Return'].std())
        }

    return final_data

Functions for Calculating RSI and MACD

In [233]:
def calculate_RSI(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the Relative Strength Index (RSI) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        period (int): The number of periods to use for RSI calculation

    Returns:
        pd.DataFrame: DataFrame with an additional 'RSI' column
    """

    df = df.copy()

    avg_gain = (df['Return'].where(df['Return'] > 0, 0)).rolling(window=window).mean()
    avg_loss = (-df['Return'].where(df['Return'] < 0, 0)).rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

def calculate_MACD(df: pd.DataFrame, short_window: int, long_window: int, signal_window: int) -> pd.DataFrame:
    """
    Calculates the Moving Average Convergence Divergence (MACD) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        short_window (int): Short-term EMA window
        long_window (int): Long-term EMA window
        signal_window (int): Signal line EMA window

    Returns:
        pd.DataFrame: DataFrame with additional EMA, MACD, MACD Signal, and MACD Histogram columns
    """

    df = df.copy()

    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()

    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    return df

Function for Calculating Both Indicators

In [234]:
def calculate_indicators(data: dict[str, dict[str, pd.DataFrame | float]], 
                         rsi_window: int = rsi_period, short_window: int = macd_short_period, long_window: int = macd_long_period, signal_window: int = macd_signal_period):
    """
    Calculates the RSI and MACD for given DataFrames.

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols
        rsi_period (int): The number of periods to use for RSI calculation (Default to 14)
        short_window (int): Short-term EMA window (Default to 12)
        long_window (int): Long-term EMA window (Default to 26)
        signal_window (int): Signal line EMA window (Default to 9)

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with EMA, MACD, MACD Signal, and MACD Histogram added
    """

    # Housekeeping
    rsi_period = rsi_window
    macd_short_period = short_window
    macd_long_period = long_window
    macd_signal_period = signal_window

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = calculate_RSI(df, rsi_period)
        df = calculate_MACD(df, macd_short_period, macd_long_period, macd_signal_period)

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }

    return final_data

Function for Filtering Missing Values after all Calculations

In [235]:
def filter_data(data: dict[str, dict[str, pd.DataFrame | float]]):
    '''
    Filters out rows of missing values for given dataFrames

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames without NaNs
    '''

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = df.loc[df.dropna().index[0]:]

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }
    return final_data

### 📈 Visualization

Functions for Plotting Prices, Moving Averages, Volume, RSI, and MACD

In [236]:
def plot_price_and_moving_average(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds traces for closing price and all moving averages to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Close' and moving average columns.
        fig (go.Figure): Plotly Figure object to which traces will be added.
        row (int): The row number of the subplot to add traces to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['Close'], name='Close', line=dict(color='black')),
        row=row, col=1
    )
    for ma_w in ma_period:
        fig.add_trace(go.Scatter(x=df.index, y=df[f'Moving_Avg_{ma_w}'], name=f'Moving Avg {ma_w}'), row=row, col=1)

def plot_volume(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds a bar chart for volume data to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Volume' column.
        fig (go.Figure): Plotly Figure object to which the volume bar chart will be added.
        row (int): The row number of the subplot to add the volume chart to.
    """

    fig.add_trace(
        go.Bar(x=df.index, y=df['Volume'], name='Volume', marker_color='gray'),
        row=row, col=1
    )

def plot_rsi(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds a line chart for RSI with threshold lines to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'RSI' column.
        fig (go.Figure): Plotly Figure object to which RSI traces will be added.
        row (int): The row number of the subplot to add the RSI plot to.
    """
    
    fig.add_trace(
        go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='orange')),
        row=row, col=1
    )
    fig.add_hline(y=70, line_dash='dash', line_color='gray', row=row, col=1)
    fig.add_hline(y=30, line_dash='dash', line_color='gray', row=row, col=1)

def plot_macd(df: pd.DataFrame, fig: go.Figure, row: int) -> None:
    """
    Adds line charts for MACD and MACD Signal, and a bar chart for MACD Histogram to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'MACD', 'MACD_Signal', and 'MACD_Hist' columns.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')),
        row=row, col=1
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['MACD_Signal'], name='MACD Signal', line=dict(color='red')),
        row=row, col=1
    )
    fig.add_trace(
        go.Bar(x=df.index, y=df['MACD_Hist'], name='MACD Hist', marker_color='green'),
        row=row, col=1
    )

Function for Plotting all the Technicals

In [237]:
def plot_technicals(df: pd.DataFrame, ticker: str) -> None:
    """
    Creates a 4-row technical analysis dashboard with subplots for price & moving averages, volume, RSI, and MACD.

    Parameters:
        df (pd.DataFrame): DataFrame containing all necessary columns for plotting technical indicators.
        ticker (str): Stock ticker symbol used for the plot title.
    """

    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    plot_price_and_moving_average(df, fig, row=1)
    plot_volume(df, fig, row=2)
    plot_rsi(df, fig, row=3)
    plot_macd(df, fig, row=4)

    fig.update_layout(
        height=1000,
        title_text=f"{ticker} Technical Analysis Dashboard",
        title_x=0.5,
        showlegend=True
    )
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=3, col=1)
    fig.update_yaxes(title_text="MACD", row=4, col=1)
    fig.update_xaxes(title_text="Date", row=4, col=1)

    return fig

Functions for Plotting Volatility vs Time, Return, Volume, RSI, and MACD

In [238]:
def plot_rolling_volatility(df: pd.DataFrame, fig: go.Figure, row: int, col: int) -> None:
    """
    Adds traces for rolling volatility over time to a specific subplot row and column.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Close' and moving average columns.
        fig (go.Figure): Plotly Figure object to which traces will be added.
        row (int): The row number of the subplot to add traces to.
        col (int): The column number fo the subplot to add to.
    """

    fig.add_trace(
        go.Scatter(x=df.index, y=df['Rolling_Vol'], name='Rolling_Vol'),
        row=row, col=col
    )

def scatter_volatility(df: pd.DataFrame, y_axis: str, fig: go.Figure, row: int, col: int) -> None:
    """
    Adds a scatterplot for volatility assessment to a specific subplot row and column.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'Volume' column.
        y_axis (str): Column to scatter for y-axis
        fig (go.Figure): Plotly Figure object to which the volume bar chart will be added.
        row (int): The row number of the subplot to add the volume chart to.
        col (int): The column number fo the subplot to add to.
    """

    fig.add_trace(
        go.Scatter(x=df['Rolling_Vol'], y=df[y_axis], mode='markers', name=y_axis),
        row=row, col=col
    )


Function for Plotting all the Volatility Measures

In [239]:
def scatterplot_volatility(df: pd.DataFrame, ticker: str):
    """
    Creates a 3-row volatility assessment dashboard with subplots for Volatility vs Time, Return, Volume, RSI, and MACD

    Parameters:
        df (pd.DataFrame): DataFrame containing all necessary columns for plotting technical indicators.
        ticker (str): Stock ticker symbol used for the plot title.
    """

    fig = make_subplots(
        rows=3, cols=2,
        specs=[[{"colspan": 2}, None],
            [{}, {}],
            [{}, {}]],                  
        row_heights=[0.4, 0.3, 0.3],
        vertical_spacing=0.1
    )

    plot_rolling_volatility(df, fig, 1, 1)
    scatter_volatility(df, 'MACD', fig, 2, 1)    
    scatter_volatility(df, 'RSI', fig, 2, 2)   
    scatter_volatility(df, 'Volume', fig, 3, 1)   
    scatter_volatility(df, 'Return', fig, 3, 2)   

    fig.update_layout(
        height=1000,
        title_text=f"{ticker} Volatility Assessment Dashboard",
        title_x=0.5,
        showlegend=True
    )
    fig.update_yaxes(title_text="Rolling Volatility", row=1)
    fig.update_xaxes(title_text="Date", row=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=2)
    fig.update_xaxes(title_text="Rolling Volatility", row=2)
    fig.update_yaxes(title_text="Volume", row=3, col=1)
    fig.update_yaxes(title_text="Return", row=3, col=2)
    fig.update_xaxes(title_text="Rolling Volatility", row=3)

    return fig

Functions for Creating Dashboard

In [240]:
def create_layout(tickers: list[str]):
    """
    Constructs and returns the full layout of the Dash stock dashboard.
    The layout includes:
        - A title header
        - Dropdown selectors for ticker and date range
        - Sliders and dropdowns for technical indicator parameters
        - Two `dcc.Store` components to hold raw and processed data
        - An interval component to trigger data loading once
        - Two graphs for price with technical overlays and volatility

    Parameters:
        tickers (List[str]): List of available stock ticker symbols to populate the dropdown.

    Returns:
        html.Div: The complete layout as a Dash HTML Div component.
    """

    return html.Div([
        html.H1(
            "Stock Dashboard",
            style={'textAlign': 'center', 'fontFamily': 'Arial, sans-serif', 'margin-bottom': '20px'}
        ),
        html.Div([
            html.Div([
                html.Label("Ticker", style={'fontFamily': 'Arial, sans-serif', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='ticker-dropdown',
                    options=[{'label': t, 'value': t} for t in tickers],
                    value=tickers[0],
                    style={'width': '300px', 'fontFamily': 'Arial, sans-serif'}
                ),
            ], style={'display': 'flex',
                      'flexDirection': 'column',
                      'alignItems': 'flex-start'
            }),
            html.Div([
                html.Label("Data Range", style={'fontFamily': 'Arial, sans-serif', 'marginBottom': '5px'}),
                dcc.DatePickerRange(
                    id='date-range-picker',
                    display_format='YYYY-MM-DD',
                    style={'width': '300px', 'fontFamily': 'Arial, sans-serif'}
                )
            ], style={'display': 'flex',
                      'flexDirection': 'column',
                      'alignItems': 'flex-start'
            }),
        ], style={'display': 'flex',
                 'justifyContent': 'center',
                 'alignItems': 'flex-start',
                 'gap': '40px',
                 'marginBottom': '30px'
        }),
        html.Div([
            html.Label("Moving Average Window 1", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='moving_average_1_window', min=2, max=250, step=1, value=50, 
                        marks={i: str(i) for i in range(5, 251, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Label("Moving Average Window 2", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='moving_average_2_window', min=2, max=250, step=1, value=200, 
                        marks={i: str(i) for i in range(5, 251, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Label("RSI Window", style={'fontFamily': 'Arial, sans-serif'}),
            dcc.Slider(id='rsi_window', min=5, max=30, step=1, value=14, 
                        marks={i: str(i) for i in range(5, 31, 5)},tooltip={'placement': 'bottom', 'always_visible': True})
        ]),
        html.Div([
            html.Div([
                html.Label("Volatility Window", style={'fontFamily': 'Arial, sans-serif'}),
                dcc.Dropdown(id='volatility-window', value=14,
                         options=[{'label': str(x), 'value': x} for x in [5, 10, 14, 20, 30, 40, 50, 60]], clearable=False,
                         style={'width': '200px', 'fontFamily': 'Arial, sans-serif'})
            ]),
            html.Div([
                html.Label("MACD Windows", style={'fontFamily': 'Arial, sans-serif'}),
                dcc.Dropdown(id='macd-windows-dropdown', value=str(macd_windows[0]['value']),
                         options=[{'label': macd_w['label'], 'value': macd_w['value']} for macd_w in macd_windows], clearable=False,
                         style={'width': '200px', 'fontFamily': 'Arial, sans-serif'})
            ])
        ], style={'display': 'flex',
                 'justifyContent': 'left',
                 'alignItems': 'flex-start',
                 'gap': '40px',
                 'marginBottom': '30px'
        }),
        dcc.Store(id='raw-data-store'),
        dcc.Store(id='data-store'),
        dcc.Interval(id='interval-once', interval=1*1000, n_intervals=0, max_intervals=1),
        html.Div([
            dcc.Graph(id='price-technicals-graph'),
            dcc.Graph(id='volatility-graph')
        ])
    ], style={
        'backgroundColor': '#ffffff',
        'minHeight': '100vh'
    })

def register_callbacks(app: Dash):
    """
    Registers the Dash callbacks for updating graphs.

    Parameters:
        app (Dash): The Dash app instance.
    """

    @app.callback(
        Output('raw-data-store', 'data'),
        Input('interval-once', 'n_intervals')
    )
    def load_all_data(_: int):
        """
        Callback to load and preprocess raw stock data once at app load.

        Parameters:
        - _ (int): Dummy input from `interval-once` used to trigger this callback once.

        Returns:
        - str: JSON-encoded dictionary where each ticker maps to its cleaned DataFrame (serialized).
        """

        raw_data = clean_data(get_stock_data(TICKERS, START_DATE, END_DATE))
        
        raw_data_json = {
            ticker: df.to_json(date_format='iso', orient='split')
            for ticker, df in raw_data.items()
        }
        
        # Serialize the whole dict to JSON string before returning
        return json.dumps(raw_data_json)

    @app.callback(
        Output('data-store', 'data'),
        Output('date-range-picker', 'min_date_allowed'),
        Output('date-range-picker', 'max_date_allowed'),
        Output('date-range-picker', 'start_date'),
        Output('date-range-picker', 'end_date'),
        Input('raw-data-store', 'data'),
        Input('moving_average_1_window', 'value'),
        Input('moving_average_2_window', 'value'),
        Input('rsi_window', 'value'),
        Input('volatility-window', 'value'),
        Input('macd-windows-dropdown', 'value'),
        State('date-range-picker', 'start_date'),
        State('date-range-picker', 'end_date'),
        prevent_initial_call=True
    )
    def update_data(raw_data: str, ma1: int, ma2: int, rsi: int, vol: int, macd_str: str, curr_start: str, curr_end: str):
        """
        Update the processed data and the allowed/selected date range in the DatePicker.

        Parameters:
        - data_json (str): JSON-encoded raw stock data.
        - ma1 (int): Window for first moving average.
        - ma2 (int): Window for second moving average.
        - rsi (int): RSI window.
        - vol (int): Volatility window.
        - macd_str (str): Comma-separated MACD window parameters (e.g., "12,26,9").
        - curr_start (str): Currently selected start date.
        - curr_end (str): Currently selected end date.

        Returns:
        - Tuple:
            - Dict[str, Dict[str, str]]: Processed and filtered stock data per ticker.
            - pd.Timestamp: Minimum available date in the data.
            - pd.Timestamp: Maximum available date in the data.
            - str: Updated start_date for DatePicker.
            - str: Updated end_date for DatePicker.
        """

        # Load raw data back into DataFrame
        raw_data_json = json.loads(raw_data)
        all_data = {
            ticker: pd.read_json(StringIO(df_json), orient='split')
            for ticker, df_json in raw_data_json.items()
        }

        # Parse MACD values from dropdown string, e.g., "12,26,9"
        macd = ast.literal_eval(macd_str)

        # Clean & compute
        all_data = calculate_measures(all_data, [ma1, ma2], vol)
        all_data = calculate_indicators(all_data, rsi, *macd)
        all_data = filter_data(all_data)

        # Output structure: {ticker: {"data": df_json}}
        output_data = {
            ticker: {"data": df['data'].to_json(date_format='iso', orient='split')}
            for ticker, df in all_data.items()
        }

        all_dates = pd.concat([df['data'].index.to_series() for df in all_data.values()])
        min_date = all_dates.min().date()
        max_date = all_dates.max().date()

        # Decide whether to update start/end dates
        new_start = str(min_date) if curr_start is None or (curr_start < str(min_date) or curr_start > str(max_date)) else curr_start
        new_end = str(max_date) if curr_end is None or (curr_end < str(min_date) or curr_end > str(max_date)) else curr_end

        return output_data, min_date, max_date, new_start, new_end

    @app.callback(
        Output('price-technicals-graph', 'figure'),
        Output('volatility-graph', 'figure'),
        Input('data-store', 'data'),
        Input('ticker-dropdown', 'value'),
        Input('date-range-picker', 'start_date'),
        Input('date-range-picker', 'end_date'),
        prevent_initial_call=True
    )
    def update_graphs(data: dict[str, dict[str, str]], ticker: str, start: str, end: str):
        """
        Update the price/technical graph and volatility graph based on selected ticker and date range.

        Parameters:
        - data (dict[str, dict[str, str]]): JSON-encoded processed data per ticker.
        - ticker (str): Selected stock ticker.
        - start (str): Selected start date.
        - end (str): Selected end date.

        Returns:
        - Tuple:
            - fig_price (plotly Figure): Price and technical indicators plot.
            - fig_volatility (plotly Figure): Volatility scatter plot.
        """

        df = pd.read_json(StringIO(data[ticker]['data']), orient='split')
        df = df[start:end]

        fig_price = plot_technicals(df, ticker)
        fig_volatility = scatterplot_volatility(df, ticker)

        return fig_price, fig_volatility

In [ ]:
app = Dash(__name__)

app.layout = create_layout(TICKERS)
register_callbacks(app)

if __name__ == '__main__':
    app.run(debug=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Error fetching data for MSFT: No objects to concatenate
Error fetching data for WMT: Length mismatch: Expected axis has 4 elements, new values have 2 elements


[*********************100%***********************]  1 of 1 completed

[*********************100%***********************]  1 of 1 completed

Error fetching data for AMZN: Length mismatch: Expected axis has 4 elements, new values have 2 elements



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
Cell In[240], line 228, in register_callbacks.<locals>.update_graphs(
    data={'AAPL': {'data': '{"columns":["Close","Close_Normalized","Volume",...4946202,2.2124173219,2.0457318808,0.1666854411]]}'}, 'AMT': {'data': '{"columns":["Close","Close_Normalized","Volume",...04861,-0.3842910621,-0.1034204741,-0.280870588]]}'}, 'JNJ': {'data': '{"columns":["Close","Close_Normalized","Volume",...5148337,7.9109579704,7.1553429191,0.7556150512]]}'}, 'JPM': {'data': '{"columns":["Close","Close_Normalized","Volume",...0512269,1.8621158419,0.9954888935,0.8666269484]]}'}, 'NVDA': {'data': '{"columns":["Close","Close_Normalized","Volume",...97979,-1.7471065887,-2.6002199944,0.8531134057]]}'}, 'PG': {'data': '{"columns":["Close","Close_Normalized","Volume",...581044,1.3752451576,1.5411542178,-0.1659090602]]}'}, 'TSLA': {'data': '{"columns":["Close","Cl